In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import LabelEncoder

2024-02-24 13:27:45.972466: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def my_custom_operation(inputs):
    squared_inputs = tf.square(inputs)
    concatenated = tf.concat([inputs, squared_inputs], axis=-1)
    reshaped_output = tf.reshape(concatenated, [-1, 4, 1])
    return reshaped_output

In [3]:
inputs = tf.keras.Input(shape=(2,1))  # Adjust the shape according to your data.
custom_op_output = my_custom_operation(inputs)
flattened_output = tf.keras.layers.Flatten()(custom_op_output)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(flattened_output)    
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss = "binary_crossentropy")

In [4]:
weights = np.array([[-0.3806], [0.3034], [-0.2139], [0.1545]])
bias = np.array([0]).reshape(1, )
bias_and_weights = []
bias_and_weights.append(weights)
bias_and_weights.append(bias)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 2, 1)]               0         []                            
                                                                                                  
 tf.math.square (TFOpLambda  (None, 2, 1)                 0         ['input_1[0][0]']             
 )                                                                                                
                                                                                                  
 tf.concat (TFOpLambda)      (None, 2, 2)                 0         ['input_1[0][0]',             
                                                                     'tf.math.square[0][0]']      
                                                                                              

In [9]:
all_data = pd.read_csv("spline-test.csv", index_col=2)
age_data = all_data['PatientAge'].values
ct_data = all_data['CT'].values #drop the /N values instead
print(age_data)
print(ct_data)

[15.         17.          8.          7.          0.66666667  6.
  8.         15.          5.         10.         10.          6.        ]
[23.95 33.65 27.28 31.22 26.8  17.34 31.08 29.66 24.8  32.4  23.6  28.4 ]


In [10]:
X = np.column_stack((age_data, ct_data))
X

array([[15.        , 23.95      ],
       [17.        , 33.65      ],
       [ 8.        , 27.28      ],
       [ 7.        , 31.22      ],
       [ 0.66666667, 26.8       ],
       [ 6.        , 17.34      ],
       [ 8.        , 31.08      ],
       [15.        , 29.66      ],
       [ 5.        , 24.8       ],
       [10.        , 32.4       ],
       [10.        , 23.6       ],
       [ 6.        , 28.4       ]])

In [11]:
Y = model.predict(X)
Y

1/1 [==============================] - 0s 13ms/step


array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [14]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.allow_custom_ops = False  # Ensuring only supported ops are allowed; set to True if custom ops are necessary.
tflite_model = converter.convert()

# Save the model.
with open('test.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/4g/z3rj5j_j781_8ckvkcy7p4840000gp/T/tmpwwq42w_b/assets


INFO:tensorflow:Assets written to: /var/folders/4g/z3rj5j_j781_8ckvkcy7p4840000gp/T/tmpwwq42w_b/assets
2024-02-24 13:38:04.621246: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-02-24 13:38:04.621258: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-02-24 13:38:04.621393: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/4g/z3rj5j_j781_8ckvkcy7p4840000gp/T/tmpwwq42w_b
2024-02-24 13:38:04.621848: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-02-24 13:38:04.621856: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/4g/z3rj5j_j781_8ckvkcy7p4840000gp/T/tmpwwq42w_b
2024-02-24 13:38:04.622899: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-02-24 13:38:04.634078: I tensorflow/cc/saved_model/loader.cc:217] Running initialization